In [9]:
import zipfile
import numpy as np
import pandas as pd
import random
with zipfile.ZipFile('hwk2_datasets.zip') as z:
    with z.open('DS1_m_0.txt','r') as f:
        DS1_m_0 = pd.read_csv(f, sep=",", header=None)
    with z.open('DS1_m_1.txt') as f:
        DS1_m_1=pd.read_csv(f, sep=",", header=None)
    with z.open('DS1_Cov.txt') as f:
        DS1_Cov=pd.read_csv(f, sep=",", header=None)

In [10]:
#data manipulation       
DS1_m_0=DS1_m_0.dropna(axis=1).values[0]
DS1_m_1=DS1_m_1.dropna(axis=1).values[0]
DS1_Cov=DS1_Cov.dropna(axis=1).values



In [11]:
#generating data
datapoint_negative=np.random.multivariate_normal(DS1_m_0,DS1_Cov,2000)
label=np.zeros((2000,1))
datapoint_negative=np.append(datapoint_negative,label,axis=1)

datapoint_positive=np.random.multivariate_normal(DS1_m_1,DS1_Cov,2000)
label2=np.full((2000,1),1)
datapoint_positive=np.append(datapoint_positive,label2,axis=1)

In [12]:
#split
dfn = pd.DataFrame(datapoint_negative)
dfp = pd.DataFrame(datapoint_positive)
randomness=random.randint(1,2000)

train_positive=dfp.sample(frac=0.6,random_state=randomness)
valid_positive=dfp.drop(train_positive.index).sample(frac=0.5,random_state=randomness)
test_positive=dfp.drop(valid_positive.index).drop(train_positive.index)

train_negative=dfn.sample(frac=0.6,random_state=randomness)
valid_negative=dfn.drop(train_negative.index).sample(frac=0.5,random_state=randomness)
test_negative=dfn.drop(valid_negative.index).drop(train_negative.index)

#dataset to report
DS1_train=train_positive.append(train_negative).sample(frac=1).reset_index(drop=True)
DS1_valid=valid_positive.append(valid_negative).sample(frac=1).reset_index(drop=True)
DS1_test=test_positive.append(test_negative).sample(frac=1).reset_index(drop=True)

DS1_train.to_csv('DS1_train.csv')
DS1_valid.to_csv('DS1_valid.csv')
DS1_test.to_csv('DS1_test.csv')


In [13]:
#Q2
#convert to np array
train_positive=np.delete(train_positive.values,20,1)
valid_positive=np.delete(valid_positive.values,20,1)
test_positive=np.delete(test_positive.values,20,1)
train_negative=np.delete(train_negative.values,20,1)
valid_negative=np.delete(valid_negative.values,20,1)
test_negative=np.delete(test_negative.values,20,1)

In [14]:
#fraction
Pi=1200/(2*1200)
#u_1 and u_2

def mu(data_matrix):
    Sum=0;
    index=0
    for x in data_matrix:
        Sum=Sum+x 
        index=index+1
    return np.array([Sum/index])
mu_1=mu(train_positive)
mu_0=mu(train_negative)
a=train_positive[0]
b=train_positive[1]
a=np.array([a])
#Sigma
def Sigma(data_matrix_0,data_matrix_1):
    mu_0=mu(data_matrix_0)
    mu_1=mu(data_matrix_1)
    temp_0=0
    temp_1=0
    sum_0=0
    sum_1=0
    for x in data_matrix_0:
        x=np.array([x])
        temp_0=x-mu_0
        temp_0=np.matmul(temp_0.T,temp_0)
        sum_0=sum_0+temp_0
    for x in data_matrix_1:
        x=np.array([x])
        temp_1=x-mu_1
        temp_1=np.matmul(temp_1.T,temp_1)
        sum_1=sum_1+temp_1
    return  (sum_0+sum_1)/(np.shape(data_matrix_0)[0]+np.shape(data_matrix_1)[0])      
sigma=Sigma(train_positive,train_negative)

sigma_inverse=np.linalg.inv(sigma)
w=np.matmul(sigma_inverse,(mu_0-mu_1).T)
part_a=-1/2*np.matmul(np.matmul(mu_0,sigma_inverse),mu_0.T)
part_b=1/2*np.matmul(np.matmul(mu_1,sigma_inverse),mu_1.T)
w_0=part_a+part_b

In [8]:
#2.1.a
import math
def classifier(x,w,w_0):
    e=math.exp(1)
    result=np.matmul(w.T,np.array([x]).T)
    result=result+w_0
    return 1/(1+math.pow(e,(-result)))

#test classifier
test=list()
tp=0
fp=0
fn=0
tn=0
#classifier(train_negative[0],w,w_0)
iteration=0
for datapoint in DS1_test.values:
    label=datapoint[20]
    datapoint=np.delete(datapoint,20,0)
    if classifier(datapoint,w,w_0)<0.5:
        if label==1:
            tp=tp+1
        else:
            fp=fp+1
    else:
        if label==1:
            fn=fn+1
        else:
            tn=tn+1
recall_1=tp/(tp+fn)
precision_1=tp/(tp+fp)
accuracy_1=(tp+tn)/(tp+tn+fp+fn)
F1_score=2*(recall_1 * precision_1)/(recall_1 + precision_1)
print("accuracy:",accuracy_1,"recall:",recall_1,", precision: ", precision_1,"F1 score:",F1_score)

accuracy: 0.96625 recall: 0.965 , precision:  0.9674185463659147 F1 score: 0.9662077596996245


In [15]:
#2.1.b
parameter={"w0":w_0,"w":w}
with open('Assignment2_260631276_2_1_b.txt', 'w') as file:
    parameter={"w0":w_0,"w":w}
    file.write(str(parameter))

In [18]:
#Q3
def distance(x1,x2):
    temp=0
    for i in range(0,len(x1)):
        temp=temp+math.pow(x1[i]-x2[i],2)
    return math.sqrt(temp)


def findksmallest(Distance,k):
    import pandas as pd
    collection=pd.DataFrame()
    for i in range(0,k):
       smallest=Distance['distance'].idxmin()
       smallest=Distance.loc[smallest]
       collection=collection.append(smallest)
       Distance=Distance.drop(Distance['distance'].idxmin())
    return collection
def makeprediction(k_smallest):
    positive=(k_smallest['label']==1).sum()
    negative=(k_smallest['label']==0).sum()
    if positive>=negative: return 1
    else: return 0  

Accuracy=list()
Recall=list()
Precision=list()
F1Score_2=list()

def knn(k,dataset,trainset):
    tp=0
    tn=0
    fn=0
    fp=0
    for datapoint in dataset.values:
        Distance=np.array([[0,0]])
        labeltest=datapoint[20]
        datapoint=np.delete(datapoint,20,0)
        for datapoint2 in trainset.values:
            labeltrain=datapoint2[20]
            datapoint2=np.delete(datapoint2,20,0)
            d=distance(datapoint,datapoint2)
            predict=np.array([[d,labeltrain]])
            Distance=np.row_stack((Distance,predict))   
        Distance = pd.DataFrame({'distance': Distance[:,0],'label': Distance[:,1]})
        Distance=Distance.drop(Distance['distance'].idxmin())
        k_smallest=findksmallest(Distance,k)
        Prediction=makeprediction(k_smallest)  
        if labeltest==1:
            if Prediction==1:
                tp=tp+1
            else:
                fn=fn+1
        else:
            if Prediction==1:
                fp=fp+1
            else:
                tn=tn+1
    recall_2=tp/(tp+fn)
    precision_2=tp/(tp+fp)
    accuracy_2=(tp+tn)/(tp+tn+fp+fn)
    F1_score_2=2*(recall_2 * precision_2)/(recall_2 + precision_2)
    print("k:",k,"accuracy:",accuracy_2,"recall:",recall_2,", precision: ", precision_2,"F1 score:",F1_score_2)
    return {"k":k,"accuracy":accuracy_2,"recall": recall_2, "precision": precision_2,"F1 score":F1_score_2}

for k in range(0,20):
    k=2*k+1
    knn(k,DS1_valid,DS1_train)


k: 1 accuracy: 0.51375 recall: 0.5075 , precision:  0.5139240506329114 F1 score: 0.5106918238993711
k: 3 accuracy: 0.53 recall: 0.5225 , precision:  0.5304568527918782 F1 score: 0.5264483627204031
k: 5 accuracy: 0.5175 recall: 0.5075 , precision:  0.5178571428571429 F1 score: 0.5126262626262627
k: 7 accuracy: 0.5175 recall: 0.4975 , precision:  0.5182291666666666 F1 score: 0.5076530612244898
k: 9 accuracy: 0.515 recall: 0.5025 , precision:  0.5153846153846153 F1 score: 0.5088607594936708
k: 11 accuracy: 0.5025 recall: 0.49 , precision:  0.5025641025641026 F1 score: 0.4962025316455696
k: 13 accuracy: 0.505 recall: 0.5025 , precision:  0.5050251256281407 F1 score: 0.5037593984962406
k: 15 accuracy: 0.52 recall: 0.515 , precision:  0.5202020202020202 F1 score: 0.5175879396984925
k: 17 accuracy: 0.5175 recall: 0.4825 , precision:  0.5188172043010753 F1 score: 0.5
k: 19 accuracy: 0.51125 recall: 0.48 , precision:  0.512 F1 score: 0.49548387096774194
k: 21 accuracy: 0.51375 recall: 0.48 , pr

In [19]:
best_k=3
knn(best_k,DS1_test,DS1_train)

k: 3 accuracy: 0.53625 recall: 0.5075 , precision:  0.5384615384615384 F1 score: 0.5225225225225224


{'k': 3,
 'accuracy': 0.53625,
 'recall': 0.5075,
 'precision': 0.5384615384615384,
 'F1 score': 0.5225225225225224}

In [20]:
#Q4
#load file
import zipfile
import numpy as np
import pandas as pd
import random
with zipfile.ZipFile('hwk2_datasets.zip') as z:
    with z.open('DS2_c1_m1.txt','r') as f:
        DS2_c1_m1 = pd.read_csv(f, sep=",", header=None)
    with z.open('DS2_c1_m2.txt') as f:
        DS2_c1_m2=pd.read_csv(f, sep=",", header=None)
    with z.open('DS2_c1_m3.txt') as f:
        DS2_c1_m3=pd.read_csv(f, sep=",", header=None)
    with z.open('DS2_c2_m1.txt','r') as f:
        DS2_c2_m1 = pd.read_csv(f, sep=",", header=None)
    with z.open('DS2_c2_m2.txt') as f:
        DS2_c2_m2=pd.read_csv(f, sep=",", header=None)
    with z.open('DS2_c2_m3.txt') as f:
        DS2_c2_m3=pd.read_csv(f, sep=",", header=None)
    with z.open('DS2_Cov1.txt','r') as f:
        DS2_Cov1 = pd.read_csv(f, sep=",", header=None)
    with z.open('DS2_Cov2.txt') as f:
        DS2_Cov2=pd.read_csv(f, sep=",", header=None)
    with z.open('DS2_Cov3.txt') as f:
        DS2_Cov3=pd.read_csv(f, sep=",", header=None)
#data manipulation
def dropnans(df):
    df=df.dropna(axis=1).values[0]
    return df
 
DS2_c1_m1=dropnans(DS2_c1_m1)
DS2_c1_m2=dropnans(DS2_c1_m2) 
DS2_c1_m3=dropnans(DS2_c1_m3) 
DS2_c2_m1=dropnans(DS2_c2_m1) 
DS2_c2_m2=dropnans(DS2_c2_m2) 
DS2_c2_m3=dropnans(DS2_c2_m3)

DS2_Cov1=DS2_Cov1.dropna(axis=1)
DS2_Cov2=DS2_Cov2.dropna(axis=1)
DS2_Cov3=DS2_Cov3.dropna(axis=1)


In [28]:
#generating data

def mixGaussian(m1,m2,m3,cov1,cov2,cov3,size):
    Size=np.random.multinomial(size, [0.10, 0.42, 0.48])
    g1 = np.random.multivariate_normal(m1,cov1,Size[0]) 
    g2 = np.random.multivariate_normal(m2,cov2,Size[1])  
    g3 = np.random.multivariate_normal(m3,cov3,Size[2]) 
    x=np.row_stack((g1,g2))  
    x=np.row_stack((x,g3)) 
    return x 
##class 1  
df1=mixGaussian(DS2_c1_m1,DS2_c1_m2,DS2_c1_m3,DS2_Cov1,DS2_Cov2,DS2_Cov3,2000)
label1=np.full((2000,1),1)
df1=np.concatenate((df1,label1),axis=1)#2000x21
##class 0
df2=mixGaussian(DS2_c2_m1,DS2_c2_m2,DS2_c2_m3,DS2_Cov1,DS2_Cov2,DS2_Cov3,2000)
label2=np.zeros((2000,1))
df2=np.concatenate((df2,label2),axis=1)#2000x21

In [29]:
#split data
randomness=random.randint(1,2000)
df1=pd.DataFrame(df1)
df2=pd.DataFrame(df2)

train_positive=df1.sample(frac=0.6,random_state=randomness)
valid_positive=df1.drop(train_positive.index).sample(frac=0.5,random_state=randomness)
test_positive=df1.drop(valid_positive.index).drop(train_positive.index)

train_negative=df2.sample(frac=0.6,random_state=randomness)
valid_negative=df2.drop(train_negative.index).sample(frac=0.5,random_state=randomness)
test_negative=df2.drop(valid_negative.index).drop(train_negative.index)
#datasets to report
DS2_train=train_positive.append(train_negative).sample(frac=1).reset_index(drop=True)
DS2_valid=valid_positive.append(valid_negative).sample(frac=1).reset_index(drop=True)
DS2_test=test_positive.append(test_negative).sample(frac=1).reset_index(drop=True)

DS2_train.to_csv('DS2_train.csv')
DS2_valid.to_csv('DS2_valid.csv')
DS2_test.to_csv('DS2_test.csv')


In [30]:
#GDA method
train_positive=np.delete(train_positive.values,20,1)
valid_positive=np.delete(valid_positive.values,20,1)
test_positive=np.delete(test_positive.values,20,1)
train_negative=np.delete(train_negative.values,20,1)
valid_negative=np.delete(valid_negative.values,20,1)
test_negative=np.delete(test_negative.values,20,1)

def mu(data_matrix):
    Sum=0;
    index=0
    for x in data_matrix:
        Sum=Sum+x 
        index=index+1
    return np.array([Sum/index])
mu_1=mu(train_positive)
mu_0=mu(train_negative)
a=train_positive[0]
b=train_positive[1]
a=np.array([a])

def Sigma(data_matrix_0,data_matrix_1):
    mu_0=mu(data_matrix_0)
    mu_1=mu(data_matrix_1)
    temp_0=0
    temp_1=0
    sum_0=0
    sum_1=0
    for x in data_matrix_0:
        x=np.array([x])
        temp_0=x-mu_0
        temp_0=np.matmul(temp_0.T,temp_0)
        sum_0=sum_0+temp_0
    for x in data_matrix_1:
        x=np.array([x])
        temp_1=x-mu_1
        temp_1=np.matmul(temp_1.T,temp_1)
        sum_1=sum_1+temp_1
    return  (sum_0+sum_1)/(np.shape(data_matrix_0)[0]+np.shape(data_matrix_1)[0])      
sigma=Sigma(train_positive,train_negative)
#get coeff
sigma_inverse=np.linalg.inv(sigma)
w=np.matmul(sigma_inverse,(mu_0-mu_1).T)
part_a=-1/2*np.matmul(np.matmul(mu_0,sigma_inverse),mu_0.T)
part_b=1/2*np.matmul(np.matmul(mu_1,sigma_inverse),mu_1.T)
w_0=part_a+part_b

In [31]:
#5.1.a
import math
def classifier(x,w,w_0):
    e=math.exp(1)
    result=np.matmul(w.T,np.array([x]).T)
    result=result+w_0
    return 1/(1+math.pow(e,(-result)))

#test classifier
test=list()
tp=0
fp=0
fn=0
tn=0
#classifier(train_negative[0],w,w_0)
iteration=0
for datapoint in DS1_test.values:
    label=datapoint[20]
    datapoint=np.delete(datapoint,20,0)
    if classifier(datapoint,w,w_0)<0.5:
        if label==1:
            tp=tp+1
        else:
            fp=fp+1
    else:
        if label==1:
            fn=fn+1
        else:
            tn=tn+1
recall_1=tp/(tp+fn)
precision_1=tp/(tp+fp)
accuracy_1=(tp+tn)/(tp+tn+fp+fn)
F1_score=2*(recall_1 * precision_1)/(recall_1 + precision_1)
print("accuracy:",accuracy_1,"recall:",recall_1,", precision: ", precision_1,"F1 score:",F1_score)

accuracy: 0.45875 recall: 0.45 , precision:  0.4580152671755725 F1 score: 0.45397225725094575


In [32]:
#5.1.b
#report parameter learnt
parameter={"w0":w_0,"w":w}
with open('Assignment2_260631276_5_1_b.txt', 'w') as file:
    parameter={"w0":w_0,"w":w}
    file.write(str(parameter))

In [34]:
#knn method
def distance(x1,x2):
    temp=0
    for i in range(0,len(x1)):
        temp=temp+math.pow(x1[i]-x2[i],2)
    return math.sqrt(temp)


def findksmallest(Distance,k):
    import pandas as pd
    collection=pd.DataFrame()
    for i in range(0,k):
       smallest=Distance['distance'].idxmin()
       smallest=Distance.loc[smallest]
       collection=collection.append(smallest)
       Distance=Distance.drop(Distance['distance'].idxmin())
    return collection
def makeprediction(k_smallest):
    positive=(k_smallest['label']==1).sum()
    negative=(k_smallest['label']==0).sum()
    if positive>=negative: return 1
    else: return 0  

Accuracy=list()
Recall=list()
Precision=list()
F1Score_2=list()


for k in range(0,20):
    k=2*k+1
    knn(k,DS2_valid,DS2_train)

k: 1 accuracy: 0.50875 recall: 0.525 , precision:  0.5084745762711864 F1 score: 0.5166051660516606
k: 3 accuracy: 0.49125 recall: 0.485 , precision:  0.4911392405063291 F1 score: 0.4880503144654088
k: 5 accuracy: 0.50125 recall: 0.4925 , precision:  0.5012722646310432 F1 score: 0.4968474148802018
k: 7 accuracy: 0.505 recall: 0.5175 , precision:  0.5048780487804878 F1 score: 0.5111111111111111
k: 9 accuracy: 0.51 recall: 0.515 , precision:  0.5099009900990099 F1 score: 0.5124378109452736
k: 11 accuracy: 0.5175 recall: 0.54 , precision:  0.5167464114832536 F1 score: 0.528117359413203
k: 13 accuracy: 0.5125 recall: 0.5225 , precision:  0.5122549019607843 F1 score: 0.5173267326732672
k: 15 accuracy: 0.52125 recall: 0.535 , precision:  0.5206812652068127 F1 score: 0.5277435265104808
k: 17 accuracy: 0.49375 recall: 0.515 , precision:  0.4940047961630695 F1 score: 0.5042839657282742
k: 19 accuracy: 0.50125 recall: 0.515 , precision:  0.5012165450121655 F1 score: 0.5080147965474723
k: 21 accur

In [36]:
best_k=21
knn(best_k,DS2_test,DS2_train)

k: 21 accuracy: 0.535 recall: 0.5075 , precision:  0.5370370370370371 F1 score: 0.5218508997429305


{'k': 21,
 'accuracy': 0.535,
 'recall': 0.5075,
 'precision': 0.5370370370370371,
 'F1 score': 0.5218508997429305}